# Notebook to experiment with ways to quantify pec fin morphology

### Import packages

In [ ]:
from ome_zarr.io import parse_url
from ome_zarr.reader import Reader
import napari
import numpy as np
from napari_animation import Animation

### Now, load an image dataset along with nucleus masks inferred using cellpose.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# set parameters
filename = "2022_12_22 HCR Sox9a Tbx5a Emilin3a_1.zarr"
readPath = "/Users/nick/Dropbox (Cole Trapnell's Lab)/Nick/pecFin/HCR_Data/built_zarr_files_small/" + filename
readPathLabels = "/Users/nick/Dropbox (Cole Trapnell's Lab)/Nick/pecFin/HCR_Data/built_zarr_files_small/" + filename + "labels"
level = 1

#############
# Main image
#############

# read the image data
store = parse_url(readPath, mode="r").store
reader = Reader(parse_url(readPath))

# nodes may include images, labels etc
nodes = list(reader())

# first node will be the image pixel data
image_node = nodes[0]
image_data = image_node.data

#############
# Labels
#############

# read the image data
store_lb = parse_url(readPathLabels, mode="r").store
reader_lb = Reader(parse_url(readPathLabels))

# nodes may include images, labels etc
nodes_lb = list(reader_lb())

# first node will be the image pixel data
label_node = nodes_lb[1]
label_data = label_node.data

# extract key image attributes
omero_attrs = image_node.root.zarr.root_attrs['omero']
channel_metadata = omero_attrs['channels']  # list of channels and relevant info
multiscale_attrs = image_node.root.zarr.root_attrs['multiscales']
axis_names = multiscale_attrs[0]['axes']
dataset_info = multiscale_attrs[0]['datasets']  # list containing scale factors for each axis

### Let's visualize the masks

In [ ]:
# Dimensions are not uniform. We want to account for this in the plot
# pull second-smallest image and experiment
im_3 = np.asarray(image_data[level])
# calculate upper resolution limit for display
res_upper = np.percentile(im_3[3, :, :, :], 99.999)
# extract useful info
scale_vec = multiscale_attrs[0]["datasets"][level]["coordinateTransformations"][0]["scale"]
channel_names = [channel_metadata[i]["label"] for i in range(len(channel_metadata))]
colormaps = [channel_metadata[i]["color"] for i in range(len(channel_metadata))]

viewer = napari.view_image(image_data[level], channel_axis=0, name=channel_names, colormap=colormaps, contrast_limits=[0, res_upper], scale=scale_vec)
labels_layer = viewer.add_labels(label_data[level], name='segmentation', scale=scale_vec)


### Those are pretty complicated...lets try looking at only centroids

In [ ]:
from skimage.measure import label, regionprops, regionprops_table
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import math

def ellipsoid_axis_lengths(central_moments):
    """Compute ellipsoid major, intermediate and minor axis length.

    Parameters
    ----------
    central_moments : ndarray
        Array of central moments as given by ``moments_central`` with order 2.

    Returns
    -------
    axis_lengths: tuple of float
        The ellipsoid axis lengths in descending order.
    """
    m0 = central_moments[0, 0, 0]
    sxx = central_moments[2, 0, 0] / m0
    syy = central_moments[0, 2, 0] / m0
    szz = central_moments[0, 0, 2] / m0
    sxy = central_moments[1, 1, 0] / m0
    sxz = central_moments[1, 0, 1] / m0
    syz = central_moments[0, 1, 1] / m0
    S = np.asarray([[sxx, sxy, sxz], [sxy, syy, syz], [sxz, syz, szz]])
    # determine eigenvalues in descending order
    eigvals = np.sort(np.linalg.eigvalsh(S))[::-1]
    return tuple([math.sqrt(20.0 * e) for e in eigvals])

label_array = np.asarray(label_data[level].compute())
regions = regionprops(label_array,image_data[level][3, :, :, :])
#regions = pd.DataFrame(regions)
# centroid_array = np.empty((len(regions), 3))
# for rg in range(len(regions['centroid-0'])):
#     centroid_array[rg, :] = np.multiply(regions[["centroid-0","centroid-1","centroid-2"]].iloc[rg], scale_vec)
    
# df = pd.DataFrame(centroid_array, columns = ['Z','Y','X'])
# print(regions.iloc[0])
# fig = px.scatter_3d(df, x='X', y='Y', z='Z',opacity=0.5)
# #fig = px.scatter_3d(pd.DataFrame(centroid_array[0:100,:]))
# fig.show()

test = ellipsoid_axis_lengths(regions[0]['moments_central'])
print(test[0])

### Pliot individual nuclei

In [ ]:
rgi = 92
im = np.asarray(regions[rgi].image_intensity)
lb = np.asarray(regions[rgi].image)
res_upper = np.percentile(im, 97)
viewer = napari.view_image(im, contrast_limits=[0, res_upper], scale=scale_vec)
labels_layer = viewer.add_labels(lb, name='segmentation', scale=scale_vec)

In [ ]:
print(len(regions))

In [ ]:
import plotly.graph_objects as go
coords = np.multiply(regions[rgi].coords, scale_vec)

# [X,Y,Z] = np.meshgrid(range(im.shape[0]),range(im.shape[1]),range(im.shape[2]))
# fig = go.Figure(data=go.Volume(
#     x=X.flatten(),
#     y=Y.flatten(),
#     z=Z.flatten(),
#     value=im.flatten()))

fig = go.Figure(data=[go.Mesh3d(x=coords[:, 2], y=coords[:, 1], z=coords[:, 0],
                   alphahull=3,
                   opacity=0.8,
                   color='cyan')])
fig.show()

In [ ]:
from scipy.spatial import Delaunay
import numpy as np
from collections import defaultdict

pos = coords
alpha = 3

"""
Compute the alpha shape (concave hull) of a set of 3D points.
Parameters:
    pos - np.array of shape (n,3) points.
    alpha - alpha value.
return
    outer surface vertex indices, edge indices, and triangle indices
"""

tetra = Delaunay(pos)
# Find radius of the circumsphere.
# By definition, radius of the sphere fitting inside the tetrahedral needs 
# to be smaller than alpha value
# http://mathworld.wolfram.com/Circumsphere.html
tetrapos = np.take(pos,tetra.vertices,axis=0)
normsq = np.sum(tetrapos**2,axis=2)[:,:,None]
ones = np.ones((tetrapos.shape[0],tetrapos.shape[1],1))
a = np.linalg.det(np.concatenate((tetrapos,ones),axis=2))
Dx = np.linalg.det(np.concatenate((normsq,tetrapos[:,:,[1,2]],ones),axis=2))
Dy = -np.linalg.det(np.concatenate((normsq,tetrapos[:,:,[0,2]],ones),axis=2))
Dz = np.linalg.det(np.concatenate((normsq,tetrapos[:,:,[0,1]],ones),axis=2))
c = np.linalg.det(np.concatenate((normsq,tetrapos),axis=2))
r = np.sqrt(Dx**2+Dy**2+Dz**2-4*a*c)/(2*np.abs(a))

# Find tetrahedrals
tetras = tetra.vertices[r<alpha,:]
# triangles
TriComb = np.array([(0, 1, 2), (0, 1, 3), (0, 2, 3), (1, 2, 3)])
Triangles = tetras[:,TriComb].reshape(-1,3)
Triangles = np.sort(Triangles,axis=1)
# Remove triangles that occurs twice, because they are within shapes
TrianglesDict = defaultdict(int)
for tri in Triangles:TrianglesDict[tuple(tri)] += 1
Triangles=np.array([tri for tri in TrianglesDict if TrianglesDict[tri] ==1])
#edges
EdgeComb=np.array([(0, 1), (0, 2), (1, 2)])
Edges=Triangles[:,EdgeComb].reshape(-1,2)
Edges=np.sort(Edges,axis=1)
Edges=np.unique(Edges,axis=0)

In [47]:
import math
import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode
from plotly.graph_objs import Mesh3d
import numpy as np

axis_lengths = np.array([11.89184616,  4.82438505,  1.49761732])

a = axis_lengths[2]
b = axis_lengths[1]
c = axis_lengths[0]

pi = math.pi
phi = np.linspace(0, 2*pi)
theta = np.linspace(-pi/2, pi/2)
phi, theta=np.meshgrid(phi, theta)
x = np.cos(theta) * np.sin(phi) * a
y = np.cos(theta) * np.cos(phi) * b
z = np.sin(theta) * c

# define a transformation matrix
R = np.array([[-0.23933507, -0.90749994,  0.34519933],
               [ 0.15574894,  0.31504453,  0.93621003],
               [ 0.95836371, -0.27783232, -0.06594095]])
T = np.zeros((4,4))
C = np.array([ 4.47448559, 192.68796498, 138.36717904])
T[0:3,0:3] = R
T[3,3] = 1
T[0:3, 3] = C

x=x.flatten() 
y=y.flatten() 
z=z.flatten()

xtf = []
ytf = []
ztf = []

for i in range(len(x)):
    xyz = np.array([z[i], y[i], x[i], 1]).reshape(4,1)
    xyz_tf = np.matmul(T,xyz)
    xyz_tf.flatten()
    xtf.append(xyz_tf[2][0])
    ytf.append(xyz_tf[1][0])
    ztf.append(xyz_tf[0][0])    

fig = go.Figure(data=[go.Mesh3d(x=xtf,
                                y=ytf,
                                z=ztf,
                                alphahull=0)])

# fig.update_layout(
#     scene = dict(
#         xaxis = dict(nticks=4, range=[-30,30]+C[0],),
#                      yaxis = dict(nticks=4, range=[-30,30]+C[1],),
#                      zaxis = dict(nticks=4, range=[-30,30]+C[2],),))

fig.show()
